### **Домашнее задание №2**
В этом задании предстоит воспроизвести и обучить одну из самых знаменитых архитектур среди CV-моделей - **AlexNet**.

![image](https://github.com/user-attachments/assets/7073f6d5-397e-4537-8b98-07a649673c0c)

Данная модель была представлена в статье [ImageNet Classification with Deep Convolutional
Neural Networks](https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf).

Подробное описание архитектуры приведено в секции статьи `3.5 Overall Architecture`.

<details>
  <summary>Формулировка задания </summary>

1. Загрузите и подготовьте данные к обучению - **2 балла**
  - В отличие от оригинальной статьи, в данном задании модель будет применятся для классификации изображений из датасета [CIFAR100](https://pytorch.org/vision/main/generated/torchvision.datasets.CIFAR100.html#torchvision.datasets.CIFAR100) - загрузите его из torchvision, разделите датасет на обучающую, валидационную и тестовую выборки.
2. Реализуйте класс модели AlexNet для решения задачи и обучите ее, продемонстрировав уменьшение функции потерь на train/validation данных, а также измерьте релевантную, на ваш взгляд метрику классификации на test-части данных - **8 баллов**


**Общее**

Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - **1 балл**


**Дополнительные баллы**

Все, описанное выше, реализовано на pytorch lightning - **2 балла**

</details>


In [1]:
# Основные библиотеки
import os
import random
import sys
import numpy as np

# Torch и связанные модули
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

# TorchVision для данных и преобразований
from torchvision import transforms, datasets
from torchvision.datasets import CIFAR100

# PyTorch Lightning
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar

# TorchMetrics для метрик
from torchmetrics import MetricCollection, Accuracy, F1Score
from torchmetrics.classification import MulticlassF1Score, MulticlassPrecision, MulticlassRecall

# Графики и визуализация
import matplotlib.pyplot as plt
import seaborn as sns

# Прогресс-бар
from tqdm.notebook import tqdm
import torch.nn as nn

In [2]:
# Фиксация
pl.seed_everything(42)

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(42)

Seed set to 42


### Подготовка данных к обучению и загрузка
**Подготовка данных с использованием `CIFAR100DataModule`:**
   - Загружаются данные CIFAR-100, применяются преобразования (горизонтальное отражение, изменение размера, нормализация).
   - Данные разделяются на тренировочную (45,000 изображений), валидационную (5,000 изображений) и тестовую выборки.
   - Созданы загрузчики данных с параметрами: батч размером 128, многопоточность.

In [3]:
class CIFAR100DataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = "../data/data_cifar", batch_size: int = 128, cutout_size: int = 8):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

        self.train_transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

        self.test_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

    def prepare_data(self):
        datasets.CIFAR100(self.data_dir, train=True, download=True)
        datasets.CIFAR100(self.data_dir, train=False, download=True)

    def setup(self, stage: str = None):
        if stage == "fit" or stage is None:
            full_train = datasets.CIFAR100(self.data_dir, train=True, transform=self.train_transform)
            self.train_data, self.val_data = random_split(full_train, [45000, 5000])

        if stage == "test" or stage is None:
            self.test_data = datasets.CIFAR100(self.data_dir, train=False, transform=self.test_transform)

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True, num_workers=4, persistent_workers=True)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size, num_workers=4, persistent_workers=True)

    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size=self.batch_size, num_workers=4, persistent_workers=True)

### Создание модели AlexNet
**Реализация модели `AlexNetCustom`:**
   - Модель основана на архитектуре AlexNet: пять свёрточных слоёв, слои MaxPooling, Dropout и три полносвязных слоя.
   - Используется инициализация весов методом Xavier Uniform для всех свёрточных и полносвязных слоёв.

**Настройка обучения:**
   - Используется функция потерь `CrossEntropyLoss` и оптимизатор `Adam` с `lr=1e-4`.
   - Логируются метрики (`Accuracy`, `F1Score`) для тренировочной, валидационной и тестовой выборок.
   - Обучение, валидация и тестирование организованы через методы `training_step`, `validation_step` и `test_step`.

In [4]:
class AlexNetCustom(pl.LightningModule):
    def __init__(self, num_classes=100):
        super(AlexNetCustom, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

        self.apply(self._init_weights)

        self.criterion = nn.CrossEntropyLoss()

        # Метрики
        self.metrics = MetricCollection([
            Accuracy(task='multiclass', num_classes=num_classes),
            F1Score(task='multiclass', num_classes=num_classes, average='weighted'),
        ])
        self.val_metrics = self.metrics.clone(prefix='val_')
        self.test_metrics = self.metrics.clone(prefix='test_')

    def forward(self, x):
        x = self.features(x)
        # Выравниваем вручную
        x = x.view(x.size(0), -1)  
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)

        # Логирование метрики точности
        self.log('train_loss', loss, prog_bar=True)
        self.log_dict(self.metrics(y_hat, y), prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)

        # Логирование метрик
        self.log('val_loss', loss, prog_bar=True)
        self.log_dict(self.val_metrics(y_hat, y), prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)

        # Логирование метрик
        self.log('test_loss', loss, prog_bar=True)
        self.log_dict(self.test_metrics(y_hat, y), prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4)

    def _init_weights(self, m):
      if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
          nn.init.xavier_uniform_(m.weight)
          if m.bias is not None:
              nn.init.zeros_(m.bias)

### Обучение модели

In [5]:
# Инициализация DataModule и модели
data_module = CIFAR100DataModule()
model = AlexNetCustom()
print(model)

AlexNetCustom(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p

1. **Логгеры:**
   - **`CSVLogger`** сохраняет метрики в CSV для постанализа.
   - **`TensorBoardLogger`** записывает логи для визуализации в TensorBoard.

2. **Сохранение модели:**
   - **`ModelCheckpoint`** сохраняет лучшую модель по `val_MulticlassF1Score` в `../logs/models/`.

3. **Ранняя остановка:**
   - **`EarlyStopping`** завершает обучение, если `val_loss` не улучшается 5 эпох подряд.

4. **Настройка тренера:**
   - Логгеры и коллбеки интегрированы для мониторинга, сохранения и управления процессом обучения.
   - Максимум 25 эпох, автоматический выбор устройства (CPU/GPU).

Результат: метрики логируются, лучшая модель сохраняется, обучение оптимизируется по ранней остановке.

In [6]:
# Логгеры
csv_logger = CSVLogger(save_dir="../logs/", name="alexnet_custom")
tb_logger = TensorBoardLogger(
    save_dir='../logs/lightning_logs',
    name='cifar100',
    version='alexnet_custom_v0.1'
)

# Callback для сохранения лучшей модели
checkpoint_callback = ModelCheckpoint(
    save_top_k=1,
    mode="max",
    monitor="val_MulticlassF1Score",
    dirpath="../logs/models/",
    filename="best_model-{epoch:02d}-{val_MulticlassF1Score:.4f}"
)

# Callback для ранней остановки
early_stopping_callback = EarlyStopping(
    monitor="val_loss",  # Мониторинг валидационных потерь
    mode="min",
    patience=5,  # Остановится, если метрика не улучшается 3 эпохи подряд
    verbose=True
)

# Настройка тренера
trainer = pl.Trainer(
    max_epochs=25,
    accelerator="auto",
    devices="auto",
    logger=[csv_logger, tb_logger],
    callbacks=[
        checkpoint_callback,
        early_stopping_callback,
        TQDMProgressBar(refresh_rate=10)
    ],
    log_every_n_steps=10,
    deterministic=True,)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [7]:
# Обучение
trainer.fit(model, datamodule=data_module)

Files already downloaded and verified
Files already downloaded and verified


/Users/anastasia/docs/ITMO/DL_Course/.venv/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/anastasia/docs/ITMO/DL_Course/logs/models exists and is not empty.

  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | features     | Sequential       | 2.5 M  | train
1 | classifier   | Sequential       | 54.9 M | train
2 | criterion    | CrossEntropyLoss | 0      | train
3 | metrics      | MetricCollection | 0      | train
4 | val_metrics  | MetricCollection | 0      | train
5 | test_metrics | MetricCollection | 0      | train
----------------------------------------------------------
57.4 M    Trainable params
0         Non-trainable params
57.4 M    Total params
229.654   Total estimated model params size (MB)
32        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 5 records. Best score: 1.720. Signaling Trainer to stop.


In [9]:
# Тестирование
trainer.test(model, datamodule=data_module)

Files already downloaded and verified
Files already downloaded and verified


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
 test_MulticlassAccuracy    0.5579000115394592
 test_MulticlassF1Score     0.5535627007484436
        test_loss           1.7912378311157227
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.7912378311157227,
  'test_MulticlassAccuracy': 0.5579000115394592,
  'test_MulticlassF1Score': 0.5535627007484436}]

### Визуализация результатов с помощью TensorBoard

#### Сравнение двух экспериментов

На графиках TensorBoard проведено сравнение текущей архитектуры и предыдущей модели. Текущая архитектура показала значительно лучшие результаты:

- **Лучшие результаты текущего эксперимента:**
  - **Test Loss:** `1.791`
  - **Test Multiclass Accuracy:** `0.558`
  - **Test Multiclass F1 Score:** `0.554`

Эти результаты почти **вдвое лучше**, чем у предыдущей модели.

#### Ключевые изменения, улучшившие качество

1. **Скорость обучения (`lr=1e-4`)**:
   - Уменьшение learning rate позволило модели обучаться более стабильно, избегая больших скачков градиента.

2. **Инициализация весов (Xavier Initialization):**
   - Использование инициализации весов методом Xavier Uniform помогло улучшить сходимость модели.

3. **Нормализация данных:**
   - Преобразование изображений с использованием нормализации (`mean=[0.5, 0.5, 0.5]` и `std=[0.5, 0.5, 0.5]`) улучшило обучение, привнеся стабильность в модель.

In [11]:
# Запуск TensorBoard сервера
from tensorboard import program
from IPython.display import IFrame

# Создаём объект TensorBoard
tb = program.TensorBoard()
tb.configure(argv=[None, "--logdir", "../logs/lightning_logs"])
# Запускаем TensorBoard сервер
url = tb.launch()  

# Отображение TensorBoard в IFrame
IFrame(src=url, width="100%", height="800px")

### Итог
Применение этих техник позволило достичь более высокой точности классификации (`0.558`) и F1 Score (`0.554`), что подтверждает эффективность оптимизаций в текущей архитектуре.